In [5]:
import websockets
import json
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [8]:
endereco_robo="dohko" # Substitua pelo endereço do robô se necessário
porta_servico="8087"
async with websockets.connect(f"ws://{endereco_robo}:{porta_servico}/wsctrl") as websocket:
    await websocket.send("ack")
    res = json.loads(await  websocket.recv())

ConnectionClosedError: code = 1006 (connection closed abnormally [internal]), no reason

In [11]:
!ping raspberrypi

PING raspberrypi (127.0.1.1) 56(84) bytes of data.
64 bytes from raspberrypi (127.0.1.1): icmp_seq=1 ttl=64 time=0.099 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=2 ttl=64 time=0.121 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=3 ttl=64 time=0.118 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=4 ttl=64 time=0.117 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=5 ttl=64 time=0.115 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=6 ttl=64 time=0.111 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=7 ttl=64 time=0.115 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=8 ttl=64 time=0.127 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=9 ttl=64 time=0.131 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=10 ttl=64 time=0.115 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=11 ttl=64 time=0.132 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=12 ttl=64 time=0.128 ms
64 bytes from raspberrypi (127.0.1.1): icmp_seq=13 ttl=64 time=0.128 ms
64 bytes from raspberr

In [ ]:
fig, ax = plt.subplots()
ax.set_aspect('equal')
ax.set_xlim(0,600)
ax.set_ylim(-300,300)
for x, y in res["coordenadas"]:
    c = mpatches.Circle((x,y), radius=12)
    ax.add_patch(c)